In [1]:
import requests
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np

In [2]:
import requests
from PIL import Image

MAX_RETRIES = 3

def send_image_for_qr_decoding(image_path):
    # File validation
    try:
        with Image.open(image_path) as img:
            if img.format not in ['JPEG', 'PNG']:
                print("Unsupported image format.")
                return
    except Exception as e:
        print(f"Error in reading the image: {e}")
        return

    retry_count = 0
    while retry_count < MAX_RETRIES:
        try:
            with open(image_path, 'rb') as image_file:
                response = requests.post(
                    'http://127.0.0.1:8000/decode-qr/',
                    files={'file': image_file}
                )
            
            decoded_info = response.json()
            if decoded_info and 'decoded_value' in decoded_info:
                print('Decoded value is:', decoded_info['decoded_value'])
                break
            else:
                print('No QR code found or error in decoding.')
                break
        except Exception as e:
            print(f"Error in request: {e}")
            retry_count += 1
            if retry_count < MAX_RETRIES:
                print(f"Retrying... ({retry_count}/{MAX_RETRIES})")
            else:
                print("Max retries reached.")



In [3]:
send_image_for_qr_decoding('../data/vpay_test/202309281459387582.jpg')

Error in request: Expecting value: line 1 column 1 (char 0)
Retrying... (1/3)
Error in request: Expecting value: line 1 column 1 (char 0)
Retrying... (2/3)
Error in request: Expecting value: line 1 column 1 (char 0)
Max retries reached.


In [10]:
def word_romanized_check(to_check_name, real_eng_name, server_url="http://127.0.0.1:8000"):
    url = f"{server_url}/name-checker/"

    data = {
        "name1": to_check_name,
        "name2": real_eng_name
    }

    try:
        response = requests.post(url, data=data)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error (response.status_code): {response.text}")
            return None
        
    except Exception as e:
        print(f"Error in sending quest: {e}")
        return None

In [11]:
word_romanized_check("ปฐวี", "patawee")

Error (response.status_code): {"detail":[{"type":"missing","loc":["body","name1"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.4/v/missing"},{"type":"missing","loc":["body","name2"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.4/v/missing"}]}


In [6]:
from lingua import Language, LanguageDetectorBuilder

languages = [Language.ENGLISH, Language.THAI]
detector = LanguageDetectorBuilder.from_languages(*languages)\
.with_minimum_relative_distance(0.9)\
.build()

In [7]:
print(detector.detect_language_of("lfdดีจ้าe is english"))

Language.THAI
